In [ ]:
#task1

import sqlite3
import pandas as pd

In [ ]:
conn_old = sqlite3.connect('netflix.sqlite')
cursor_old = conn_old.cursor()

In [ ]:
netflix_data = pd.read_sql_query('SELECT * FROM netflix_titles', conn_old)

In [ ]:
conn_new = sqlite3.connect('task1.sqlite')
cursor_new = conn_new.cursor()

In [ ]:
cursor_new.execute('''
    CREATE TABLE IF NOT EXISTS actors (
        actor_id INTEGER PRIMARY KEY AUTOINCREMENT,
        actor_name TEXT NOT NULL
    )
''')

In [ ]:
cursor_new.execute('''
    CREATE TABLE IF NOT EXISTS movies (
        movie_id INTEGER PRIMARY KEY AUTOINCREMENT,
        show_id TEXT NOT NULL,
        type TEXT,
        title TEXT,
        director TEXT,
        country TEXT,
        date_added TEXT,
        release_year INTEGER,
        rating TEXT,
        duration TEXT,
        listed_in TEXT,
        description TEXT
    )
''')

In [ ]:
cursor_new.execute('''
    CREATE TABLE IF NOT EXISTS actor_movie (
        actor_name TEXT,
        title TEXT,
        PRIMARY KEY (actor_name, title),
        FOREIGN KEY (actor_name) REFERENCES actors (actor_name),
        FOREIGN KEY (title) REFERENCES movies (title)
    )
''')

In [ ]:
actors_data = pd.DataFrame({'actor_name': netflix_data['cast'].str.split(', ').explode().unique()})
actors_data.to_sql('actors', conn_new, if_exists='replace', index_label='actor_id')

In [ ]:
movies_data = netflix_data[['show_id', 'type', 'title', 'director', 'country', 'date_added',
                            'release_year', 'rating', 'duration', 'listed_in', 'description']].copy()
movies_data.to_sql('movies', conn_new, if_exists='replace', index=False)

In [ ]:
actor_movie_data = pd.DataFrame(columns=['actor_name', 'title'])
for _, row in netflix_data.iterrows():
    actor_names = row['cast'].split(', ')
    title = row['title']
    actor_movie_data = pd.concat([actor_movie_data, pd.DataFrame({'actor_name': actor_names, 'title': [title] * len(actor_names)})], ignore_index=True)

actor_movie_data.to_sql('actor_movie', conn_new, if_exists='replace', index=False)

In [ ]:
conn_old.close()
conn_new.commit()
conn_new.close()

In [ ]:
#task2

import sqlite3
conn = sqlite3.connect('task1.sqlite')
cursor = conn.cursor()

In [ ]:
query = '''
    SELECT a1.actor_name AS Actor1, a2.actor_name AS Actor2, COUNT(*) AS MoviesTogether
    FROM actor_movie a1
    JOIN actor_movie a2 ON a1.title = a2.title AND a1.actor_name < a2.actor_name
    GROUP BY a1.actor_name, a2.actor_name
    ORDER BY MoviesTogether DESC
    LIMIT 1;
'''

In [ ]:
cursor.execute(query)

In [ ]:
result = cursor.fetchone()

In [ ]:
print(f"Наиболее часто работающая друг с другом пара актеров: {result[0]} и {result[1]}")

In [ ]:
conn.close()